# Example: Estimating a Large VAR Term Structure Model with Macroeconomic Variables

This notebook provides an example of how to use the TermStructureModels.jl package to estimate a large vector autoregression of the yield curve with macroeconomic variables. The procedure of the algorithm is as follows.

1. Data and Settings
2. Optimization of Hyperparameters
3. Estimation
4. Statistical Inferences: Yield Curve Interpolation, Posterior Distribution of Parameters, Term Premiums
5. Scenario Analysis

You need to download two data files(["current.csv"](https://github.com/econPreference/TermStructureModels.jl/blob/main/examples/LargeVAR_Yields_Macros/current.csv) and ["LW_monthly.xlsx"](https://github.com/econPreference/TermStructureModels.jl/blob/main/examples/LargeVAR_Yields_Macros/LW_monthly.xlsx)). The data files and this notebook must be in the same directory. "current.csv" contains macroeconomic data, and "LW_monthly.xlsx" contains bond yields data.


## Data and Settings

### Packages

First, load the necessary packages. If you need to install the packages, run

```julia
using Pkg
Pkg.activate(@__DIR__)

Pkg.add("TermStructureModels")
Pkg.add(["CSV", "Dates", "DataFrames", "XLSX", "JLD2"])

Pkg.instantiate()
Pkg.precompile()
```

After installing the packages, run


In [1]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
Pkg.precompile()

using TermStructureModels
using CSV, Dates, DataFrames, XLSX, JLD2

### Data

**Note:** You do not have to understand this Data section. Load the data using your preferred method. The essential variables that need to be set in this section are `tau_n`, `rho`, `macros`, and `yields`.

`date_start` and `date_end` define the start and end dates of the dataset. `tau_n` is a Vector that contains maturities of the yield data. `data_loading` is a function to load yield and macro data.

`sdate` is also a function you do not need to use. This function is merely a convenience for fetching observations at the desired points in time. For example, if you want to load macro data from May to July 2001, you can do so by executing `macros[sdate(2001,5):sdate(2001,7), :]`.

The package requires `yields::Matrix` and `macros::Matrix`, but, in this code, we have `yields::DataFrame` and `macros::DataFrame`. It may be more convenient for you to set your data as struct `Matrix`. Also, note that the first columns of the `yields` and `macros` are date variables.


In [ ]:
date_start = Date("1987-01-01", "yyyy-mm-dd") |> x -> x - Month(18 + 2)
date_end = Date("2022-12-01", "yyyy-mm-dd")

tau_n = [1; 3; 6; 9; collect(12:6:60); collect(72:12:120)]
function data_loading(; date_start, date_end, tau_n)

    ## Macro data
    raw_fred = CSV.File("current.csv") |> DataFrame |> x -> x[314:769, :]
    raw_fred = [Date.(raw_fred[:, 1], DateFormat("mm/dd/yyyy")) raw_fred[:, 2:end]]
    raw_fred = raw_fred[findall(x -> x == yearmonth(date_start), yearmonth.(raw_fred[:, 1]))[1]:findall(x -> x == yearmonth(date_end), yearmonth.(raw_fred[:, 1]))[1], :]

    excluded = ["FEDFUNDS", "CP3Mx", "TB3MS", "TB6MS", "GS1", "GS5", "GS10", "TB3SMFFM", "TB6SMFFM", "T1YFFM", "T5YFFM", "T10YFFM", "COMPAPFFx", "AAAFFM", "BAAFFM"]
    macros = raw_fred[:, findall(x -> !(x ∈ excluded), names(raw_fred))]
    idx = ones(Int, 1)
    for i in axes(macros[:, 2:end], 2)
        if sum(ismissing.(macros[:, i+1])) == 0
            push!(idx, i + 1)
        end
    end
    macros = macros[:, idx]
    excluded = ["W875RX1", "IPFPNSS", "IPFINAL", "IPCONGD", "IPDCONGD", "IPNCONGD", "IPBUSEQ", "IPMAT", "IPDMAT", "IPNMAT", "IPMANSICS", "IPB51222S", "IPFUELS", "HWIURATIO", "CLF16OV", "CE16OV", "UEMPLT5", "UEMP5TO14", "UEMP15OV", "UEMP15T26", "UEMP27OV", "USGOOD", "CES1021000001", "USCONS", "MANEMP", "DMANEMP", "NDMANEMP", "SRVPRD", "USTPU", "USWTRADE", "USTRADE", "USFIRE", "USGOVT", "AWOTMAN", "AWHMAN", "CES2000000008", "CES3000000008", "HOUSTNE", "HOUSTMW", "HOUSTS", "HOUSTW", "PERMITNE", "PERMITMW", "PERMITS", "PERMITW", "NONBORRES", "DTCOLNVHFNM", "AAAFFM", "BAAFFM", "EXSZUSx", "EXJPUSx", "EXUSUKx", "EXCAUSx", "WPSFD49502", "WPSID61", "WPSID62", "CPIAPPSL", "CPITRNSL", "CPIMEDSL", "CUSR0000SAC", "CUSR0000SAS", "CPIULFSL", "CUSR0000SA0L2", "CUSR0000SA0L5", "DDURRG3M086SBEA", "DNDGRG3M086SBEA", "DSERRG3M086SBEA"]
    push!(excluded, "CMRMTSPLx", "RETAILx", "HWI", "UEMPMEAN", "CLAIMSx", "AMDMNOx", "ANDENOx", "AMDMUOx", "BUSINVx", "ISRATIOx", "BUSLOANS", "NONREVSL", "CONSPI", "S&P: indust", "S&P div yield", "S&P PE ratio", "M1SL", "BOGMBASE")
    macros = macros[:, findall(x -> !(x ∈ excluded), names(macros))]
    macros = [macros[:, 1] Float64.(macros[:, 2:end])]
    rename!(macros, Dict(:x1 => "date"))
    raw_macros = deepcopy(macros)

    rho = Vector{Float64}(undef, size(macros[:, 2:end], 2))
    is_percent = fill(false, size(macros[:, 2:end], 2))
    idx_diff = Vector{Float64}(undef, size(macros[:, 2:end], 2))
    logmacros = similar(macros[:, 2:end] |> Array)
    for i in axes(macros[:, 2:end], 2) # i'th macro variable (excluding date)
        logmacros[:, i] = 100log.(macros[:, i+1])

        if names(macros[:, 2:end])[i] ∈ ["CUMFNS", "UNRATE", "AAA", "BAA"]
            is_percent[i] = true
        end

        if names(macros[:, 2:end])[i] ∈ ["AAA", "BAA"]
            macros[2:end, i+1] = macros[2:end, i+1] - macros[1:end-1, i+1]
            rho[i] = 0.0
            idx_diff[i] = 1
        elseif names(macros[:, 2:end])[i] ∈ ["CUMFNS", "UNRATE"]
            rho[i] = 1.0
            idx_diff[i] = 0
        elseif names(macros[:, 2:end])[i] ∈ ["CES0600000007", "VIXCLSx"]
            macros[:, i+1] = log.(macros[:, i+1]) |> x -> 100 * x
            rho[i] = 1.0
            idx_diff[i] = 0
        else
            macros[2:end, i+1] = log.(macros[2:end, i+1]) - log.(macros[1:end-1, i+1]) |> x -> 1200 * x
            rho[i] = 0.0
            idx_diff[i] = 1
        end
    end

    raw_macros = raw_macros[3:end, :]
    macros = macros[3:end, :]
    logmacros = logmacros[3:end, :]
    mean_macros = mean(macros[:, 2:end] |> Array, dims=1)[1, :]
    macros[:, 2:end] .-= mean_macros'

    ## Yield data
    raw_yield = XLSX.readdata("LW_monthly.xlsx", "Sheet1", "A293:DQ748") |> x -> [Date.(string.(x[:, 1]), DateFormat("yyyymm")) convert(Matrix{Float64}, x[:, tau_n.+1])] |> x -> DataFrame(x, ["date"; ["Y$i" for i in tau_n]])
    yields = raw_yield[findall(x -> x == yearmonth(date_start), yearmonth.(raw_yield[:, 1]))[1]:findall(x -> x == yearmonth(date_end), yearmonth.(raw_yield[:, 1]))[1], :]
    yields = yields[3:end, :]

    yields = [Date.(string.(yields[:, 1]), DateFormat("yyyy-mm-dd")) Float64.(yields[:, 2:end])]
    rename!(yields, Dict(:x1 => "date"))

    return rho, is_percent, idx_diff, logmacros, raw_macros, macros, mean_macros, yields
end
rho, is_percent, idx_diff, logmacros, raw_macros, macros, mean_macros, yields = data_loading(; date_start, date_end, tau_n)

sdate(yy, mm) = findall(x -> x == Date(yy, mm), macros[:, 1])[1]

### Settings

Currently, the package requires the number of spanned factors to be exactly three(`dQ = 3`). However, anticipating the possibility of relaxing this constraint in the future, the number of factors has been generalized through `dimQ()`. `dP` is the sum of `dQ` and the number of macro variables. `medium_tau` and `std_kQ_infty` are set based on prior belief.

`iteration` is the total length of the MCMC, and `burnin` is the size of burn-in. `TP_tau = [24, 120]` indicates that the term premiums of the 2-year(24 months) and 10-year(120 months) bonds are estimated. `TP_tau` should be a `Vector`. So, if you want to estimate only a single maturity term premium, set `TP_tau` as a `Vector` of length `1`.


In [ ]:
dQ = dimQ()
dP = size(macros, 2) - 1 + dQ
medium_tau = collect(36:42)
std_kQ_infty = 0.2

iteration = 25_000
burnin = 5_000
TP_tau = [24, 120]

**Tip.** If you want to reduce computation time, modify the `iteration` and `burnin` as follows.

```julia
iteration = 2_000
burnin = 500
```


### Write scenarios

`scenario_TP = [12, 24, 60, 120]` indicates that term premiums of one, two, five, and ten-year bonds are predicted. `scenario_horizon` is the forecasting horizon. `gen_scene(idx_case)` generates `Vector{Scenario}` based on the scenarios.


In [ ]:
scenario_TP = [12, 24, 60, 120]
scenario_horizon = 60
function gen_scene(idx_case)

    if idx_case == 1
        scene = Vector{Scenario}(undef, 36)
        for h in 1:36
            combs = zeros(1, dP - dQ + length(tau_n))
            vals = [0.0]
            scene[h] = Scenario(combinations=deepcopy(combs), values=deepcopy(vals))
        end

        combs = [1 zeros(1, dP - dQ + length(tau_n) - 1)]
        vals = [5.1]
        scene[12] = Scenario(combinations=deepcopy(combs), values=deepcopy(vals))

        combs = [1 zeros(1, dP - dQ + length(tau_n) - 1)]
        vals = [4.1]
        scene[24] = Scenario(combinations=deepcopy(combs), values=deepcopy(vals))

        combs = [1 zeros(1, dP - dQ + length(tau_n) - 1)]
        vals = [3.1]
        scene[end] = Scenario(combinations=deepcopy(combs), values=deepcopy(vals))
        return scene
    elseif idx_case == 2
        scene = Vector{Scenario}(undef, 10)
        VIX_path = raw_macros[sdate(2008, 9):sdate(2009, 6), end]
        for h in 1:10
            combs = zeros(1, dP - dQ + length(tau_n))
            vals = zeros(size(combs, 1))

            combs[1, end] = 1.0
            vals[1] = 100log(VIX_path[h]) - mean_macros[end]
            scene[h] = Scenario(combinations=deepcopy(combs), values=deepcopy(vals))
        end
        return scene
    end
end

## [Optimization of Hyperparameters](https://econpreference.github.io/TermStructureModels.jl/dev/estimation/#Step-1.-Tuning-Hyperparameters)

Hyperparameters are optimized by running


In [ ]:
tuned, opt = tuning_hyperparameter(Array(yields[:, 2:end]), Array(macros[:, 2:end]), tau_n, rho; std_kQ_infty, medium_tau)
p = tuned.p

`yields[:, 2:end]` and `macros[:, 2:end]` erase the first columns(date variables). `Array` transforms `DataFrame` to `Matrix`.


## [Estimation](https://econpreference.github.io/TermStructureModels.jl/dev/estimation/#Step-2.-Sampling-the-Posterior-Distribution-of-Parameters)

To get posterior samples of parameters, run


In [ ]:
saved_params, acceptPrMH = posterior_sampler(Array(yields[:, 2:end]), Array(macros[:, 2:end]), tau_n, rho, iteration, tuned; medium_tau, std_kQ_infty)

After the MCMC simulation, do the burn-in procedure. And then, erase posterior samples that do not satisfy the stationary condition. These works can be done by


In [ ]:
saved_params = saved_params[burnin+1:end]
iteration = length(saved_params)

saved_params, Pr_stationary = erase_nonstationary_param(saved_params)
iteration = length(saved_params)

In this code, `iteration` represents the number of remaining posterior samples. Therefore, it needs to be adjusted when some samples are discarded.

Lastly, [calculate the inefficiency factors](https://econpreference.github.io/TermStructureModels.jl/dev/estimation/#Diagnostics-for-MCMC) by running


In [ ]:
ineff = ineff_factor(saved_params)

## Statistical Inferences

To get [posterior samples of parameters](https://econpreference.github.io/TermStructureModels.jl/dev/inference/#Inference-for-Parameters) in the term structure model, run


In [ ]:
reduced_params = reducedform(saved_params, Array(yields[:, 2:end]), Array(macros[:, 2:end]), tau_n)

[Yield curve interpolations](https://econpreference.github.io/TermStructureModels.jl/dev/inference/#Yield-Curve-Interpolation) can be done by


In [ ]:
saved_latent_params = latentspace(saved_params, Array(yields[:, 2:end]), tau_n)
fitted_yields = fitted_YieldCurve(collect(1:tau_n[end]), saved_latent_params)

The first input of `fitted_YieldCurve` specifies the maturities for which you want to compute the fitted yields.

Lastly, [the term premium](https://econpreference.github.io/TermStructureModels.jl/dev/inference/#Term-Premiums) is calculated by


In [ ]:
iter_sub = (ineff[1] |> maximum, ineff[2], ineff[3] |> maximum, ineff[4] |> maximum, ineff[5] |> maximum, ineff[6] |> maximum) |> maximum |> ceil |> Int
saved_TP = term_premium(TP_tau, tau_n, saved_params[1:iter_sub:end], Array(yields[:, 2:end]), Array(macros[:, 2:end]))[1]

`iter_sub` is the ceiling of the maximum inefficiency factor, and `saved_params[1:iter_sub:end]` is the subsampling. Considering computational cost, posterior samples of the term premium are drawn using only a subset of the posterior samples.

There's no need to keep `saved_TP` in memory, so it's beneficial to store it on a storage device and free up memory space. This can be done with the following code.


In [12]:
JLD2.save("TP.jld2", "TP", saved_TP)
TP = nothing
# GC.gc() # It's better to let the garbage collector work automatically, so we remove this line.

This process is referred to as "Garbage Collection". `saved_TP` stored in "TP.jld2" can be loaded through

```julia
saved_TP = JLD2.load("TP.jld2")["TP"]
```


## [Forecasting](https://econpreference.github.io/TermStructureModels.jl/dev/scenario/#Forecasting)

### [Baseline Forecasts](https://econpreference.github.io/TermStructureModels.jl/dev/scenario/#Baseline-Forecasts)


In [ ]:
projections = conditional_expectation([], scenario_TP, scenario_horizon, saved_params[1:iter_sub:end], Array(yields[:, 2:end]), Array(macros[:, 2:end]), tau_n; mean_macros)

JLD2.save("uncond_scenario.jld2", "projections", projections)
projections = nothing
# GC.gc() # It's better to let the garbage collector work automatically, so we remove this line.

This also performs the garbage collection. `projections` stored in "uncond_scenario.jld2" can be loaded through

```julia
projections = JLD2.load("uncond_scenario.jld2")["projections"]
```


### [Scenario Forecasts](https://econpreference.github.io/TermStructureModels.jl/dev/scenario/#Scenario-Forecasts)


In [ ]:
for i in 1:2
    projections = conditional_expectation(gen_scene(i), scenario_TP, scenario_horizon, saved_params[1:iter_sub:end], Array(yields[:, 2:end]), Array(macros[:, 2:end]), tau_n; mean_macros)

    JLD2.save("scenario$i.jld2", "projections", projections)
    projections = nothing
    # GC.gc() # It's better to let the garbage collector work automatically, so we remove this line.
end

The last three lines are the garbage collection. You can load saved `projections` through `JLD2.load`.
